## 4. Padrão Recomendado

**Processamento pesado:** Spark (TBs de dados)  
**Analytics rápidas:** DuckDB (GBs a TBs)  
**Transformações:** Polars (performance) ou Pandas (simplicidade)  
**Interoperabilidade:** Arrow (zero-copy)

In [ ]:
import duckdb
import pyarrow as pa
import pyarrow.parquet as pq

con = duckdb.connect()

# Delta → Arrow (zero-copy quando possível)
arrow_table = con.execute("SELECT * FROM delta_scan('./sales_partitioned') LIMIT 1000").arrow()

print(f"✓ Arrow Table:")
print(f"  Schema: {arrow_table.schema}")
print(f"  Rows: {arrow_table.num_rows:,}")

# Arrow → Parquet
pq.write_table(arrow_table, 'output.parquet', compression='snappy')

# Arrow → Pandas (zero-copy)
df_pandas = arrow_table.to_pandas()
print(f"\n✓ Convertido para Pandas: {len(df_pandas)} rows")

## 3. DuckDB + Arrow (Zero-Copy)

Arrow permite transferência zero-copy entre sistemas.

In [ ]:
import duckdb
import polars as pl

con = duckdb.connect()

# Ler Delta → Polars
df_polars = con.execute("SELECT * FROM delta_scan('./sales_partitioned') LIMIT 1000").pl()

# Processar com Polars (lazy)
result = (
    df_polars
    .lazy()
    .filter(pl.col('amount') > 100)
    .with_columns([
        (pl.col('amount') * 1.1).alias('amount_with_tax'),
        pl.col('year').cast(pl.Int32)
    ])
    .group_by(['year', 'country'])
    .agg([
        pl.col('amount').sum().alias('total_amount'),
        pl.col('order_id').count().alias('order_count')
    ])
    .collect()
)

print("✓ Polars lazy execution:")
print(result)

## 2. DuckDB + Polars

Polars oferece performance superior para transformações.

In [ ]:
import duckdb
import pandas as pd
from deltalake import write_deltalake

con = duckdb.connect()

# Ler Delta → Pandas
df = con.execute("SELECT * FROM delta_scan('./sales_partitioned') LIMIT 1000").df()
print(f"✓ Loaded {len(df)} rows to Pandas")
print(df.head())

# Processar com Pandas
df['amount_with_tax'] = df['amount'] * 1.1
aggregated = df.groupby('country')['amount'].sum().reset_index()

# Escrever Pandas → Delta
write_deltalake("./processed_sales", aggregated, mode="overwrite")
print("\n✓ Pipeline Pandas completo")

## 1. DuckDB + Pandas

In [ ]:
# Instalação
%pip install duckdb deltalake pyarrow pandas polars -q

# Capítulo 09: Integração Python e Spark

Integrar DuckDB com Pandas, Polars, Arrow e PySpark.

# Capitulo 09 Integracao Python Spark

Notebook gerado automaticamente a partir do código fonte python.
